<a href="https://colab.research.google.com/github/swetha-rana/Assignment-2/blob/main/PartA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import requests
url = 'https://storage.googleapis.com/wandb_datasets/nature_12K.zip'
r = requests.get(url, allow_redirects=True)
open('nature_12K.zip', 'wb').write(r.content)
!wget 'https://storage.googleapis.com/wandb_datasets/nature_12K.zip'
!unzip nature_12K.zip

In [ ]:
!pip install wandb

In [ ]:
import wandb
wandb.init(project="CS6910_Assignment2", entity="swe-rana")
from wandb.keras import WandbCallback

In [ ]:
import pandas as pd
import os
import keras
import tensorflow as tf
from keras.models import Sequential,Input,Model
from keras.layers import Dense, Dropout, Flatten,BatchNormalization
from keras.layers import Conv2D, MaxPooling2D
#from tensorflow.keras.optimizers import Adam
import matplotlib.pyplot as plt

#from tensorflow.keras.metrics import categorical_crossentropy
# from keras.layers.normalization import BatchNormalization

from tensorflow.keras.preprocessing.image import ImageDataGenerator

def data(image_size,augment_data,batch_size):
      home_path = "/content/inaturalist_12K"
      train_path = os.path.join(home_path,'train')
      test_path = os.path.join(home_path,'val')
      if augment_data == True:
        train_data = ImageDataGenerator(rotation_range=10, width_shift_range=0.2, height_shift_range=0.2, zoom_range=0.2, horizontal_flip=True,brightness_range=[0.2,1.2],validation_split=0.1) # brightness
        test_data = ImageDataGenerator(rotation_range=10, width_shift_range=0.2, height_shift_range=0.2, zoom_range=0.2, horizontal_flip=True,brightness_range=[0.2,1.2]) # brightness

      else:
        train_data = ImageDataGenerator(rescale=1./255, validation_split=0.1)
        test_data = ImageDataGenerator(rescale=1./255)


      train_generator = train_data.flow_from_directory(directory= train_path,target_size=(image_size, image_size), batch_size=batch_size,color_mode="rgb",class_mode="categorical",subset = "training")
      val_generator = train_data.flow_from_directory(directory= train_path,target_size=(image_size, image_size), batch_size=batch_size,color_mode="rgb",class_mode="categorical",subset = "validation")
      test_generator = test_data.flow_from_directory(directory= test_path,target_size=(image_size, image_size), batch_size=batch_size,color_mode="rgb",class_mode="categorical")
      return train_generator,val_generator,test_generator

def CNN_model(image_size,kernel_size,num_filters,filter_org,dropout,batch_norm,epochs,dense_size,lr):
            if filter_org == "same":
                filters = [num_filters,num_filters,num_filters,num_filters,num_filters]
            if filter_org == "doubling":
                filters = [num_filters,num_filters*2,num_filters*4,num_filters*8,num_filters*16]
            if filter_org == "halving":
                filters = [num_filters,int(num_filters/2),int(num_filters/4),int(num_filters/8),int(num_filters/16)]

            nature_model = Sequential()
            nature_model.add(Conv2D(filters[0], kernel_size=(kernel_size, kernel_size),activation='relu',input_shape=(image_size,image_size,3)))
            if batch_norm == True:
                          print("batch_norm")
                          nature_model.add(BatchNormalization())
            nature_model.add(MaxPooling2D(2, 2))

            nature_model.add(Conv2D(filters[1], kernel_size=(kernel_size, kernel_size),activation='relu',input_shape=(image_size,image_size,3)))
            if batch_norm == True:
                          nature_model.add(BatchNormalization())
            nature_model.add(MaxPooling2D(2, 2)) 
            nature_model.add(BatchNormalization())

            nature_model.add(Conv2D(filters[2], kernel_size=(kernel_size, kernel_size),activation='relu',input_shape=(image_size,image_size,3)))
            if batch_norm == True:
                          nature_model.add(BatchNormalization())
            nature_model.add(MaxPooling2D(2, 2))  

            nature_model.add(Conv2D(filters[3], kernel_size=(kernel_size, kernel_size),activation='relu',input_shape=(image_size,image_size,3)))
            if batch_norm == True:
                          nature_model.add(BatchNormalization())
            nature_model.add(MaxPooling2D(2, 2)) 

            nature_model.add(Conv2D(filters[4], kernel_size=(kernel_size, kernel_size),activation='relu',input_shape=(image_size,image_size,3)))
            if batch_norm == True:
                          nature_model.add(BatchNormalization())
            nature_model.add(MaxPooling2D(2, 2))             
            nature_model.add(Flatten())
            nature_model.add(Dense(dense_size, activation='relu'))
            nature_model.add(Dropout(dropout))

            
            nature_model.add(Dense(10, activation='softmax'))
            return nature_model

#hist = nature_model.fit(train_generator,epochs=2,validation_data=val_generator)
#nature_model.compile(optimizer='adam', loss='categorical_crossentropy',metrics=['accuracy'])
#nature_train = nature_model.fit(x = train_generator, validation_data = val_generator, epochs = 2)



In [ ]:
def setRunName(image_size,kernel_size,num_filters, filter_org, augment_data, dropout,batch_size,dense_size, batch_norm):
    
    augment_data_options = {True: "Y", False: "N"}
    batch_norm_options = {True: "Y", False: "N"}

    run_name = "_".join(["img",str(image_size),"ker",str(kernel_size),"num", str(num_filters), "org", str(filter_org), "aug", augment_data_options[augment_data],
                      "drop", str(dropout), "bat",str(batch_size),"den",str(dense_size),"norm", batch_norm_options[batch_norm]])
    
    return run_name


In [ ]:
    def train():

        config_defaults = {
            "image_size" :200,
            "kernel_size" : 5,
            "num_filters": 32,
            "filter_org": "same",
            "augment_data": False,
            "dropout": 0.3,
            "batch_norm": False,
            "epochs": 10,
            "batch_size" : 256,
            "dense_size": 64,
            "lr": 0.001
        }

        wandb.init(config=config_defaults)
        config = wandb.config
        wandb.run.name = setRunName(config.image_size,config.kernel_size,config.num_filters, config.filter_org, config.augment_data, config.dropout,config.batch_size,config.dense_size, config.batch_norm)
       
        train_gen, val_gen, test_gen = data(image_size=config.image_size,augment_data=config.augment_data,batch_size=config.batch_size)
        model = CNN_model(image_size=config.image_size,kernel_size=config.kernel_size,num_filters=config.num_filters, filter_org=config.filter_org,dropout=config.dropout, batch_norm=config.batch_norm,epochs=config.epochs,dense_size=config.dense_size,lr=config.lr)
        model.compile(optimizer="adam", loss="categorical_crossentropy", metrics="categorical_accuracy")
        model.fit(train_gen, epochs=config.epochs, validation_data=val_gen, callbacks=[WandbCallback()])
        model.summary()


In [ ]:
#Set up a sweep config
sweep_config = {
    "name": "Assignment_2_partA_all_params",
    "description": "Assignment_2_partA_all_params",
    "method": "bayes",
    "metric" : {
        "name" : "val_categorical_accuracy",
        "goal" : "maximize"
    },
    "project": "CS6910_Assignment2",
    "parameters": {
        "num_filters": {
            "values": [32,64]
        },
        "filter_org": {
            "values": ["same","doubling","halving"]
        },
        "augment_data": {
            "values": [True,False]
        },
        "kernel_size" : {
            "values" : [3,4,5]
        },
        "image_size" : {
            "values" :[200,256,512]
        },
        "batch_size":{
            "values":[200,256,400]
        },
        "dropout": {
            "values": [0.2,0.3]
        },
        "batch_norm": {
            "values": [True,False]
        },
        "dense_size": {
            "values": [32,64]
        },
        "lr": {
            "values": [0.001]
        },
        "epochs": {
            "values": [10]
        }
    }
}

# creating the sweep


In [ ]:
#wandb.agent(sweep_id, function=train)
sweep_id = "p2xljnch"
wandb.agent(sweep_id,train,entity="swe-rana",project="CS6910_Assignment2")